<a href="https://colab.research.google.com/github/Shubhk0/personalpyprojects/blob/main/Aish_AI_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> # First, let's install the required libraries. You can run this cell to install them:
# !pip install google-generativeai SpeechRecognition gtts ipywidgets



In [ ]:
!pip install google-generativeai SpeechRecognition gtts ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.6 MB/s eta 0:00:00


In [ ]:
!sudo apt-get install portaudio19-dev python-all-dev python3-all-dev
!pip install PyAudio==0.2.12 --force-reinstall


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0 libpython-all-dev libpython2-dev
  libpython2-stdlib libpython2.7 libpython2.7-dev libpython2.7-minimal
  libpython2.7-stdlib libpython3-all-dev python-all python2 python2-dev
  python2-minimal python2.7 python2.7-dev python2.7-minimal python3-all
Suggested packages:
  portaudio19-doc python2-doc python-tk python2.7-doc binfmt-support
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 libpython-all-dev libpython2-dev
  libpython2-stdlib libpython2.7 libpython2.7-dev libpython2.7-minimal
  libpython2.7-stdlib libpython3-all-dev portaudio19-dev python-all
  python-all-dev python2 python2-dev python2-minimal python2.7 python2.7-dev
  python2.7-minimal python3-all python3-all-dev
0 upgraded, 21 newly installed, 0 to remove and 49 not upgraded.
Need to get 8,166 kB of archives.
After

In [ ]:
# First, let's install the required libraries. You can run this cell to install them:
# !pip install google-generativeai SpeechRecognition gtts ipywidgets

import os
import speech_recognition as sr
from gtts import gTTS
from IPython.display import Audio, display
from google.generativeai import GenerativeModel, configure
import ipywidgets as widgets

class JupyterVoiceAssistant:
    def __init__(self):
        # Configure the Gemini API
        configure(api_key=os.environ.get("GEMINI_API_KEY"))

        # Initialize the Gemini model
        self.model = GenerativeModel('gemini-pro')

        # Initialize speech recognizer
        self.recognizer = sr.Recognizer()

        # Initialize conversation history
        self.conversation_history = []

        # Define assistant's personality
        self.personality = (
            "You are a friendly and knowledgeable AI assistant named Mitra. "
            "You speak Hindi fluently but can understand and respond in English too. "
            "You have a warm personality and occasionally use Indian cultural references. "
            "You're helpful but also encourage users to think for themselves. "
            "Keep your responses concise and suitable for voice conversation."
        )

    def listen(self):
        with sr.Microphone() as source:
            print("Listening...")
            audio = self.recognizer.listen(source)
            try:
                text = self.recognizer.recognize_google(audio, language="hi-IN")
                print(f"You said: {text}")
                return text
            except sr.UnknownValueError:
                print("Sorry, I didn't catch that. Could you please repeat?")
                return None
            except sr.RequestError:
                print("Sorry, there was an error with the speech recognition service.")
                return None

    def speak(self, text):
        tts = gTTS(text=text, lang='hi')
        tts.save("response.mp3")
        display(Audio("response.mp3", autoplay=True))
        os.remove("response.mp3")

    def generate_response(self, user_input):
        prompt = f"{self.personality}\n\nConversation history:\n"
        for entry in self.conversation_history[-5:]:
            prompt += f"{entry['role']}: {entry['content']}\n"
        prompt += f"Human: {user_input}\nMitra:"

        response = self.model.generate_content(prompt)
        return response.text

    def chat(self):
        self.speak("नमस्ते! मैं मित्र हूं, आपका AI सहायक। आप मुझसे किसी भी विषय पर बात कर सकते हैं।")
        print("Say 'बंद करो' or 'quit' to end the conversation.")

        while True:
            user_input = self.listen()
            if user_input is None:
                continue

            if user_input.lower() in ["बंद करो", "अलविदा", "bye", "quit", "exit"]:
                self.speak("अच्छा, फिर मिलते हैं! आपका दिन मंगलमय हो।")
                break

            self.conversation_history.append({"role": "Human", "content": user_input})

            response = self.generate_response(user_input)
            print(f"Mitra: {response}")
            self.speak(response)

            self.conversation_history.append({"role": "Mitra", "content": response})

# Create a button to start the conversation
start_button = widgets.Button(description="Start Conversation")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        assistant = JupyterVoiceAssistant()
        assistant.chat()

start_button.on_click(on_button_clicked)

# Display the button and output
display(start_button, output)

Button(description='Start Conversation', style=ButtonStyle())

Output()